In [2]:
import re
import numpy as np

In [4]:
# Download for here : https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

import gensim.downloader as api

model = api.load("word2vec-google-news-300")
word_vectors = model.wv 

[=================================================-] 99.9% 1661.5/1662.8MB downloaded


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [29]:
words_ = model.index2word #word2vec orders words in decreasing order of frequency in the training corpus.

In [30]:
word_rank = {}

for i,word in enumerate(words_):
  word_rank[word] = i

In [34]:
class Autocorrect:

  def __init__(self, freq_rank_matrix):
    if freq_matrix is not None:
      self.word_freq_rank_matrix = freq_rank_matrix
    else:
      raise Exception("Empty freq matrix")
    
  def __words(self,text): 
    return re.findall(r'\w+', text.lower())

  def P(self, word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - self.word_freq_rank_matrix.get(word, 0)

  def correction(self,word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

  def candidates(self,word): 
      "Generate possible spelling corrections for word."
      return (self.__known([word]) or self.__known(EditDist1(word)) or self.__known(EditDist2(word)) or [word])

  def __known(self,words): 
      "The subset of `words` that appear in the dictionary of word_freq_rank_matrix."
      return set(w for w in words if w in word_freq_rank_matrix)

  def EditDist1(self,word):
      "All edits that are one edit away from `word`."
      letters    = 'abcdefghijklmnopqrstuvwxyz'
      splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
      deletes    = [L + R[1:]               for L, R in splits if R]
      transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
      replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
      inserts    = [L + c + R               for L, R in splits for c in letters]
      return set(deletes + transposes + replaces + inserts)

  def EditDist2(self,word): 
      "All edits that are two edits away from `word`."
      return (e2 for e1 in EditDist1(word) for e2 in EditDist1(e1))
    

In [36]:
ac = Autocorrect(word_rank)

In [46]:
ac.correction('tippi')

'tippy'

// Scribe

In [47]:
import pandas as pd
word_freqrank_df =pd.DataFrame(word_rank.items(), columns=['word', 'freq'])
word_freqrank_df.head(10)

In [49]:
word_freqrank_df.to_csv("word2vec-google-news-300-index2word-wordfreqrank.csv",index=False)
from google.colab import files
files.download("word2vec-google-news-300-index2word-wordfreqrank.csv")